# emotiCAPTCHA: An Experiment with Nari Labs' Dia TTS Model

**Author:** [0x5844](https://github.com/0x5844)

**Date:** May 7, 2025

**Experiment Goal:** To demonstrate a novel audio-based CAPTCHA system ("*emotiCAPTCHA*") utilizing the Dia Text-to-Speech model for generating dynamic audio challenges. This notebook showcases the concept and a basic implementation.


**How emotiCAPTCHA Works:**
1. A short audio dialogue is generated using the Dia TTS API, featuring a non-verbal cue (e.g., laugh, sigh) from one of the speakers.
2. The user listens to the audio.
3. The user must correctly identify:
    a. Which speaker performed the non-verbal action.
    b. The non-verbal action itself (selected via an emoji).
    c. The general topic of the conversation. (Comprehension)

## 1. Setup and Dependencies

**A. Dependencies:**
Make sure you have the following Python libraries installed. You can install them using pip:

In [1]:
!pip install numpy requests soundfile emoji ipython

**B. Hugging Face API Token:**
This experiment uses the Dia TTS model via a Hugging Face Inference API.
- You'll need a Hugging Face API token.
- It's recommended to set your token as an environment variable named `HF_API_KEY`.
- Alternatively, you can paste your token directly into the `HF_TOKEN` variable in the configuration cell below (not recommended for sharing publicly).

**C. Temporary Output Directory:**
The script may save temporary audio files (e.g., downloaded from the API before playback) in a `tmp_emoticaptcha_output` directory created in the same location as this notebook.

In [2]:
## Step 2. Imports
import json
import os
import random
import tempfile
import urllib.request
import uuid
from pathlib import Path
from typing import Any, Dict, List, Optional, Tuple

import numpy as np
import requests
import soundfile as sf
import emoji

# For playing audio directly in the notebook
from IPython.display import Audio, display, Markdown

## 2. Configuration
These are the global settings for the EmotiCAPTCHA system.

In [3]:
## Step 3. Config
HF_TOKEN: Optional[str] = "hf_xxxxxxxxxxxxxxxxx" # CHANGE ME
API_URL: str = "https://router.huggingface.co/fal-ai/fal-ai/dia-tts"
HEADERS: Dict[str, str] = {
    "Authorization": f"Bearer {HF_TOKEN}",
    "Content-Type": "application/json"
}

BASE_DIR: Path = Path.cwd()
OUTPUT_DIR: Path = BASE_DIR / "tmp_emoticaptcha_output"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True) # Ensure it exists

Scenario = Dict[str, Any]

if not HF_TOKEN:
    display(Markdown("<font color='red'>**Warning:** Using a default or placeholder Hugging Face API Token. API calls might fail. Please set your `HF_API_KEY` environment variable or update `HF_TOKEN` in the cell above.</font>"))
else:
    print("Hugging Face API Token loaded.")

print(f"Base directory for outputs (if any): {BASE_DIR}")
print(f"Temporary output directory: {OUTPUT_DIR}")

Hugging Face API Token loaded.
Base directory for outputs (if any): /content
Temporary output directory: /content/tmp_emoticaptcha_output


## 3. Definitions for EmotiCAPTCHA

**A. Emoji Mapping for Non-Verbal Cues:**
This dictionary maps the textual representation of non-verbal cues (as used in Dia's input) to emojis for user selection.

In [4]:
SUPPORTED_NON_VERBAL_CUES_EMOJI: Dict[str, str] = {
    "laughs": "😂",
    "sighs": "😔",
    "coughs": "🤧",
    "gasps": "😲",
    "clears throat": "🗣️",
    "burps": "🤢",
    "cries": "😢",
    "whistles": "😗"
}
print("Supported non-verbal cues and their emojis:")
for cue, emo_char in SUPPORTED_NON_VERBAL_CUES_EMOJI.items():
    print(f"- {emoji.emojize(emo_char)} {cue}")

Supported non-verbal cues and their emojis:
- 😂 laughs
- 😔 sighs
- 🤧 coughs
- 😲 gasps
- 🗣️ clears throat
- 🤢 burps
- 😢 cries
- 😗 whistles


**B. CAPTCHA Scenarios:**
Each scenario defines a dialogue, the correct non-verbal cue, the speaker performing it, and distractor topics.
- `id`: A unique identifier (less critical without caching, but good for tracking).
- `topic`: The correct topic of the conversation.
- `context`: A brief description of the scenario.
- `dialogue`: The text input for the Dia TTS model.
- `non_verbal_speaker`: The speaker tag (`S1` or `S2`) that performs the cue.
- `non_verbal_cue`: The textual representation of the cue.
- `speaker_names`: Fictional names for `S1` and `S2`.
- `topic_options`: A list of topics for the multiple-choice question.

In [5]:
SCENARIOS: List[Scenario] = [
    {
        "id": "s1",
        "topic": "Tech product",
        "context": "Alex and Taylor discussing a new AI model.",
        "dialogue": "[S1] EmotiCAPTCHA is an incredible new security system using audio. [S2] You can completely customize the challenges too! [S1] Wow. Amazing. (laughs) [S2] Try it on your website today!",
        "non_verbal_speaker": "S1",
        "non_verbal_cue": "laughs",
        "speaker_names": {"S1": "Alex", "S2": "Taylor"},
        "topic_options": ["Tech product", "Weather forecast", "Movie review", "Sports game"]
    },
    {
        "id": "s2",
        "topic": "Transportation",
        "context": "Morgan and Riley experiencing train delays.",
        "dialogue": "[S1] This train is always late! I can't believe this keeps happening. [S2] (sighs) I know, it's ridiculous. We're going to miss the meeting again. [S1] Should we just get a cab? [S2] Yes, let's do that.",
        "non_verbal_speaker": "S2",
        "non_verbal_cue": "sighs",
        "speaker_names": {"S1": "Morgan", "S2": "Riley"},
        "topic_options": ["Transportation", "Grocery shopping", "Holiday plans", "Book club"]
    },
    {
        "id": "s3",
        "topic": "Emergency",
        "context": "Jordan and Casey responding to a fire alarm.",
        "dialogue": "[S1] Oh my god! Is that a fire alarm? What's the procedure? [S2] Everyone needs to evacuate immediately! [S1] Where's the nearest exit? [S2] (coughs) This way, through the smoke! Follow me!",
        "non_verbal_speaker": "S2",
        "non_verbal_cue": "coughs",
        "speaker_names": {"S1": "Jordan", "S2": "Casey"},
        "topic_options": ["Emergency", "Birthday party", "School project", "Gardening tips"]
    },
    {
        "id": "s4",
        "topic": "Comedy",
        "context": "Quinn telling a joke to Avery.",
        "dialogue": "[S1] Why don't scientists trust atoms? [S2] I don't know, why? [S1] Because they make up everything! [S2] (laughs) That's terrible but I love it!",
        "non_verbal_speaker": "S2",
        "non_verbal_cue": "laughs",
        "speaker_names": {"S1": "Quinn", "S2": "Avery"},
        "topic_options": ["Comedy", "Historical event", "Cooking recipe", "Car maintenance"]
    },
    {
        "id": "s5",
        "topic": "Surprise gift",
        "context": "Jamie giving Dakota an unexpected present.",
        "dialogue": "[S1] I got something for you. Open it! [S2] What is it? [S1] Just open it! [S2] (gasps) Oh my goodness! How did you find this? I've been looking everywhere!",
        "non_verbal_speaker": "S2",
        "non_verbal_cue": "gasps",
        "speaker_names": {"S1": "Jamie", "S2": "Dakota"},
        "topic_options": ["Surprise gift", "Job interview", "Pet adoption", "Learning a language"]
    },
    {
        "id": "s6",
        "topic": "Meeting preparation",
        "context": "Harper and Rowan getting ready for a presentation.",
        "dialogue": "[S1] Are you ready for your part of the presentation? [S2] Almost. Just need to review my notes. (clears throat) Ok, I think I'm good now. [S1] You'll do great. Remember to speak slowly.",
        "non_verbal_speaker": "S2",
        "non_verbal_cue": "clears throat",
        "speaker_names": {"S1": "Harper", "S2": "Rowan"},
        "topic_options": ["Meeting preparation", "Concert review", "Home renovation", "Fitness routine"]
    },
     {
        "id": "s7",
        "topic": "Restaurant experience",
        "context": "Blake and Elliot dining at a new restaurant.",
        "dialogue": "[S1] How's your food? Mine is incredible! [S2] It's amazing but they gave me way too much. (burps) Oh! Excuse me! That was embarrassing. [S1] (laughs) At least that means you enjoyed it!",
        "non_verbal_speaker": "S2",
        "non_verbal_cue": "burps",
        "speaker_names": {"S1": "Blake", "S2": "Elliot"},
        "topic_options": ["Restaurant experience", "Art exhibition", "Political debate", "Camping trip"]
    },
    {
        "id": "s8",
        "topic": "Emotional support",
        "context": "Parker comforting Jamie after a tough day.",
        "dialogue": "[S1] I had the worst day at work. [S2] (cries) I'm so sorry to hear that. Do you want to talk about it? [S1] No, I just need some ice cream and a movie. [S2] That sounds perfect.",
        "non_verbal_speaker": "S2",
        "non_verbal_cue": "cries",
        "speaker_names": {"S1": "Parker", "S2": "Jamie"},
        "topic_options": ["Emotional support", "Travel plans", "Fitness goals", "Cooking class"]
    },
    {
        "id": "s9",
        "topic": "Music concert",
        "context": "Casey and Taylor discussing a recent concert.",
        "dialogue": "[S1] Did you see the band last night? [S2] Yes! (whistles) They were amazing! [S1] I can't believe how good they sounded live. [S2] I know, right? I can't wait for their next show!",
        "non_verbal_speaker": "S2",
        "non_verbal_cue": "whistles",
        "speaker_names": {"S1": "Casey", "S2": "Taylor"},
        "topic_options": ["Music concert", "Cooking competition", "Fashion show", "Tech conference"]
    },
]
print(f"{len(SCENARIOS)} scenarios loaded.")

9 scenarios loaded.


## 4. Core Logic: Helper Functions

**A. API Interaction:**
This function handles querying the Dia TTS API (via Hugging Face) to get the audio on demand.

In [6]:
def query_audio_generation_on_demand(scenario: Scenario) -> Optional[Tuple[np.ndarray, int]]:
    """Queries the Dia TTS API for audio generation on demand."""
    scenario_id = scenario['id'] # Used for logging/tracking
    dialogue_text = scenario['dialogue']

    if not HF_TOKEN:
        display(Markdown("<font color='red'>**API Error:** Hugging Face API token (HF_API_KEY) is not properly set. Cannot query API.</font>"))
        return None

    print(f"Generating audio via API for scenario: {scenario_id} (Text: \"{dialogue_text[:50]}...\")")
    payload = {"text": dialogue_text}

    try:
        response = requests.post(API_URL, headers=HEADERS, json=payload, timeout=90)
        response.raise_for_status()
        api_response = response.json()

        audio_data: Optional[np.ndarray] = None
        sampling_rate: Optional[int] = None

        # Process API response (same logic as before for different formats)
        if isinstance(api_response, dict) and 'audio' in api_response:
            audio_info = api_response['audio']
            if isinstance(audio_info, dict) and 'url' in audio_info:
                audio_url = audio_info['url']
                api_sampling_rate = audio_info.get('sampling_rate')

                # Download from URL to a temporary file
                # This temporary file will be deleted after use by soundfile if not explicitly kept
                with tempfile.NamedTemporaryFile(suffix='.wav', delete=True, dir=OUTPUT_DIR) as temp_file_obj:
                    temp_download_path = Path(temp_file_obj.name)
                    # print(f"Downloading audio from: {audio_url} to {temp_download_path}") # Optional: for verbose logging
                    urllib.request.urlretrieve(audio_url, temp_download_path)
                    audio_data, read_sr = sf.read(temp_download_path, dtype='float32')
                # temp_download_path is now deleted if delete=True was effective

                sampling_rate = int(api_sampling_rate) if api_sampling_rate else read_sr
                if api_sampling_rate and int(api_sampling_rate) != read_sr:
                     print(f"API/File SR Warning: API SR ({api_sampling_rate}Hz) and file SR ({read_sr}Hz) mismatch. Using file SR.")
                     sampling_rate = read_sr
                # print(f"Audio downloaded and read with SR: {sampling_rate} Hz.") # Optional

            elif isinstance(audio_info, (list, np.ndarray)) and 'sampling_rate' in api_response: # Direct data
                audio_data = np.array(audio_info, dtype=np.float32)
                sampling_rate = int(api_response['sampling_rate'])
                # print(f"Audio data received directly with SR: {sampling_rate} Hz.") # Optional

        elif isinstance(api_response, (list, tuple)) and len(api_response) == 2: # Older format
            audio_data_list, sr_int_val = api_response
            if isinstance(audio_data_list, (list, np.ndarray)) and isinstance(sr_int_val, (int, float)):
                audio_data = np.array(audio_data_list, dtype=np.float32)
                sampling_rate = int(sr_int_val)
                # print(f"Audio data (list format) received directly with SR: {sampling_rate} Hz.") # Optional

        if audio_data is not None and sampling_rate is not None:
            return audio_data, sampling_rate # No caching step
        else:
            display(Markdown(f"<font color='red'>**API Error:** Unexpected or incomplete API response format for {scenario_id}. Response: `{api_response}`</font>"))
            return None

    except requests.exceptions.Timeout:
        display(Markdown(f"<font color='red'>**API Error:** Request timed out for {scenario_id}.</font>"))
        return None
    except requests.exceptions.HTTPError as http_err:
        error_message = f"<font color='red'>**API Error:** HTTP error for {scenario_id}: {http_err}.</font>"
        if http_err.response is not None:
            error_message += f" <font color='red'>Response: `{http_err.response.text}`</font>"
        display(Markdown(error_message))
        return None
    except requests.exceptions.RequestException as req_err:
        display(Markdown(f"<font color='red'>**API Error:** Network request failed for {scenario_id}: {req_err}.</font>"))
        return None
    except (json.JSONDecodeError, KeyError, ValueError, sf.LibsndfileError) as proc_err: # Added soundfile error
        error_message = f"<font color='red'>**Processing Error:** Failed to process API response or audio for {scenario_id}: {proc_err}.</font>"
        if 'response' in locals() and response is not None: # Check if response object exists
            error_message += f" <font color='red'>Problematic API Response Content (if applicable): `{response.text}`</font>"
        display(Markdown(error_message))
        return None
    except Exception as e: # Catch-all for other unexpected errors
        display(Markdown(f"<font color='red'>**Unexpected Error:** Error processing audio for {scenario_id}: {e}.</font>"))
        return None

print("On-demand API interaction function defined.")

On-demand API interaction function defined.


**B. Audio Playback (Jupyter Specific):**
This function plays audio directly within the Jupyter Notebook output cell.

In [7]:
def play_audio_in_notebook(audio_data: Optional[np.ndarray], sampling_rate: Optional[int], scenario_id: str = "") -> None:
    """Plays audio data using IPython.display.Audio."""
    if audio_data is not None and sampling_rate is not None:
        print(f"Playing audio for scenario: {scenario_id} (SR: {sampling_rate} Hz)")
        display(Audio(data=audio_data, rate=sampling_rate))
    else:
        display(Markdown("<font color='orange'>**Audio Playback:** No audio data to play.</font>"))

print("Audio playback function for notebook defined.")

Audio playback function for notebook defined.


**C. CAPTCHA Logic:**
These functions manage the presentation of the challenge and verification of user answers.

In [8]:
def present_challenge() -> Optional[Scenario]:
    """Selects a random scenario, generates its audio on demand, and presents the challenge."""
    if not SCENARIOS:
        display(Markdown("<font color='red'>**Error:** No scenarios defined. Cannot present challenge.</font>"))
        return None

    selected_scenario = random.choice(SCENARIOS)
    display(Markdown("--- \n ## EmotiCAPTCHA Challenge"))
    display(Markdown(f"**Scenario Context:** {selected_scenario['context']}"))

    # Call the on-demand audio generation function
    audio_result = query_audio_generation_on_demand(selected_scenario)

    if audio_result:
        audio_data, sampling_rate = audio_result
        play_audio_in_notebook(audio_data, sampling_rate, selected_scenario['id'])

        speaker_names_display = ", ".join(selected_scenario['speaker_names'].values())
        display(Markdown(f"**Listen carefully.** The speakers are: **{speaker_names_display}**."))
        display(Markdown(f"Your task: Identify who performs a non-verbal action, what that action is (select the emoji), and the main topic."))

        return selected_scenario
    else:
        display(Markdown("<font color='red'>**Critical Error:** Failed to generate or retrieve audio. Cannot proceed.</font>"))
        return None

def get_user_answers(scenario: Scenario) -> Tuple[str, str, str]:
    """Prompts the user for their answers to the CAPTCHA challenge."""
    speaker_names = list(scenario['speaker_names'].values())
    speaker_prompt = f"Which speaker performed the non-verbal action? ({'/'.join(speaker_names)}): "
    user_answer_speaker = input(speaker_prompt).strip()

    display(Markdown("\n**What non-verbal action did they perform? Select the number for the action:**"))
    cue_options_list = list(SUPPORTED_NON_VERBAL_CUES_EMOJI.items())
    options_md = ""
    for i, (cue_text, cue_emoji_char) in enumerate(cue_options_list):
        options_md += f"{i + 1}. {emoji.emojize(cue_emoji_char)} {cue_text}<br>"
    display(Markdown(options_md))

    user_answer_cue_str = ""
    while True:
        try:
            choice_input = input(f"Enter the number for the non-verbal action (1-{len(cue_options_list)}): ")
            cue_choice_idx = int(choice_input) - 1
            if 0 <= cue_choice_idx < len(cue_options_list):
                user_answer_cue_str = cue_options_list[cue_choice_idx][0]
                break
            else:
                print(f"Invalid choice. Please enter a number between 1 and {len(cue_options_list)}.")
        except ValueError:
            print("Invalid input. Please enter a number.")

    display(Markdown("\n**What was the main topic of the conversation?**"))
    topic_options = scenario['topic_options']
    topic_options_md = ""
    for i, option in enumerate(topic_options):
        topic_options_md += f"{i + 1}. {option}<br>"
    display(Markdown(topic_options_md))

    user_answer_topic_str = ""
    while True:
        try:
            choice_input = input(f"Enter the number for the topic (1-{len(topic_options)}): ")
            choice_idx = int(choice_input) - 1
            if 0 <= choice_idx < len(topic_options):
                user_answer_topic_str = topic_options[choice_idx]
                break
            else:
                print(f"Invalid choice. Please enter a number between 1 and {len(topic_options)}.")
        except ValueError:
            print("Invalid input. Please enter a number.")

    return user_answer_speaker, user_answer_cue_str, user_answer_topic_str


def verify_answers(scenario: Scenario, user_speaker: str, user_cue: str, user_topic: str) -> bool:
    """Verifies the user's answers against the correct answers from the scenario."""
    correct_speaker_code = scenario['non_verbal_speaker']
    correct_speaker_name = scenario['speaker_names'][correct_speaker_code]
    correct_cue = scenario['non_verbal_cue']
    correct_topic = scenario['topic']

    is_speaker_correct = user_speaker.lower() == correct_speaker_name.lower()
    is_cue_correct = user_cue.lower() == correct_cue.lower()
    is_topic_correct = user_topic.lower() == correct_topic.lower()

    if is_speaker_correct and is_cue_correct and is_topic_correct:
        display(Markdown("<br><font color='green' size='+1'>✅ **CAPTCHA Verification: Success!**</font>"))
        return True
    else:
        display(Markdown("<br><font color='red' size='+1'>❌ **CAPTCHA Verification: Failed.**</font>"))
        if not is_speaker_correct:
            display(Markdown(f"&nbsp;&nbsp;&nbsp;&nbsp;- Incorrect speaker. Expected: '{correct_speaker_name}', You said: '{user_speaker}'"))
        if not is_cue_correct:
            correct_cue_emoji = SUPPORTED_NON_VERBAL_CUES_EMOJI.get(correct_cue, "")
            user_cue_emoji = SUPPORTED_NON_VERBAL_CUES_EMOJI.get(user_cue, "")
            expected_display = f"'{emoji.emojize(correct_cue_emoji)} {correct_cue}'" if correct_cue_emoji else f"'{correct_cue}'"
            user_display = f"'{emoji.emojize(user_cue_emoji)} {user_cue}'" if user_cue_emoji else f"'{user_cue}'"
            display(Markdown(f"&nbsp;&nbsp;&nbsp;&nbsp;- Incorrect non-verbal cue. Expected: {expected_display}, You selected: {user_display}"))
        if not is_topic_correct:
            display(Markdown(f"&nbsp;&nbsp;&nbsp;&nbsp;- Incorrect topic. Expected: '{correct_topic}', You said: '{user_topic}'"))
        return False

print("CAPTCHA logic functions defined.")

CAPTCHA logic functions defined.


## 5. Running the EmotiCAPTCHA Experiment

The cell below will initiate the EmotiCAPTCHA challenge.
- Audio will be fetched from the API each time you run this cell.
- Listen to the audio that plays.
- Answer the questions that appear below the audio player in the input prompts.

In [9]:
def run_emoticaptcha_experiment():
    display(Markdown("="*30))
    display(Markdown(f"🎙️  **Welcome to the EmotiCAPTCHA Experiment!** {emoji.emojize(':microphone:')} 🎙️"))
    display(Markdown("="*30))
    display(Markdown("You will hear a short dialogue between two speakers. One speaker will perform a non-verbal action."))
    display(Markdown("Your task is to identify:"))
    display(Markdown("  1. Which speaker performed the action."))
    display(Markdown(f"  2. What the non-verbal action was (select the corresponding {emoji.emojize(':grinning_face:')} emoji)."))
    display(Markdown("  3. The general topic of their conversation."))
    display(Markdown("---"))

    # OUTPUT_DIR is created in Cell 5, no need for a separate setup_directories call here

    current_scenario = present_challenge()

    if current_scenario:
        user_speaker_ans, user_cue_ans, user_topic_ans = get_user_answers(current_scenario)

        if verify_answers(current_scenario, user_speaker_ans, user_cue_ans, user_topic_ans):
            display(Markdown(f"<br><font color='green' size='+1'>🎉 **Congratulations! You have successfully passed the EmotiCAPTCHA!** {emoji.emojize(':party_popper:')}</font> 🎉"))
        else:
            display(Markdown(f"<br><font color='red' size='+1'>😔 **EmotiCAPTCHA failed.**</font> {emoji.emojize(':pensive_face:')}"))
    else:
        display(Markdown("<font color='red'>**Application Error:** Could not present the CAPTCHA challenge. Check console/output for API errors or setup issues.</font>"))

# --- Execute the Experiment ---
run_emoticaptcha_experiment()

==============================

🎙️  **Welcome to the EmotiCAPTCHA Experiment!** 🎤 🎙️

==============================

You will hear a short dialogue between two speakers. One speaker will perform a non-verbal action.

Your task is to identify:

...


**What was the main topic of the conversation?**

1. Comedy<br>2. Historical event<br>3. Cooking recipe<br>4. Car maintenance<br>

Enter the number for the topic (1-4): 1


<br><font color='green' size='+1'>✅ **CAPTCHA Verification: Success!**</font>

<br><font color='green' size='+1'>🎉 **Congratulations! You have successfully passed the EmotiCAPTCHA!** 🎉</font> 🎉

## 6. Conclusion & Further Thoughts

This notebook demonstrates a basic implementation of emotiCAPTCHA using Nari Labs' Dia TTS model [HERE](https://huggingface.co/nari-labs/Dia-1.6B), with audio fetched on-demand for each challenge. This approach ensures fresh audio every time but may be slower due to network latency and API response times for each execution.

**Potential areas for further development:**
- Integration into a web application.
- More diverse scenario generation.
- Adaptive difficulty.
- Robustness against automated attacks.
- User experience improvements.